In [1]:
import arcpy as ap
import os
import cmath
from collections import defaultdict
from collections import Counter


# function to return coordinates from arc point object
def coordinates_of(point_object):
    x = round(point_object.X,5)
    y = round(point_object.Y,5)

    return x,y


def ten_meters_as_percentage(of_a_line):
    of_line_length = of_a_line.length
    percentage_of_line = float(10)/float(of_line_length)
    
    return percentage_of_line


# function to return coordinates of the point x meters from the start or end of a line:
def point_ten_meters_from_the_start(of_a_line):
    point_10m_from_start = of_a_line.positionAlongLine(ten_meters_as_percentage(of_a_line), True).firstPoint
    
    return coordinates_of(point_10m_from_start)
    
    
# function to return coordinates of the point x meters from the start or end of a line:
def point_ten_meters_from_the_end(of_a_line):
    point_10m_from_end = of_a_line.positionAlongLine(1 -ten_meters_as_percentage(of_a_line), True).firstPoint
    
    return coordinates_of(point_10m_from_end)


# function to calculate the angle between two sets of coordinatesdef angle_between(first_point_coordinates, last_point_coordinates):
def angle_between(point_a_coordinates, point_b_coordinates): 
    x1, y1 = float(point_a_coordinates[0]), float(point_a_coordinates[1])
    x2, y2 = float(point_b_coordinates[0]), float(point_b_coordinates[1])

    polarcoords = cmath.polar(complex(x2 - x1, y2 - y1))
    
    if math.degrees(polarcoords[1]) <= 90:
        degrees = 90 - math.degrees(polarcoords[1])
    else:
        degrees = 450 - math.degrees(polarcoords[1])

    return round(degrees, 1)


def angle_of_first_10m(of_a_line):
    the_lines = of_a_line
    first_point = coordinates_of(the_lines.firstPoint)
    point_10m_from_start = point_ten_meters_from_the_start(of_a_line)
    angle = angle_between(first_point, point_10m_from_start)
   
    return angle
    
    
def angle_of_last_10m(of_a_line):
    the_lines = of_a_line
    last_point = coordinates_of(the_lines.lastPoint)
    point_10m_from_end = point_ten_meters_from_the_end(of_a_line)
    angle = angle_between(last_point, point_10m_from_end)
    
    return angle

In [2]:
local_shapefiles_directory = r'C:\Users\mcintyrei\Desktop\Projects\Conflating_Networks' + '\\network_shapefiles'
local_geodatabase = r'C:\Users\mcintyrei\Desktop\Projects\Conflating_Networks' + '\\working_files.gdb'

#local_shapefiles_directory = os.getcwd() + '\\network_shapefiles'
#local_geodatabase = os.getcwd() + '\\working_files.gdb'

ap.env.workspace = local_shapefiles_directory
network_shapefiles = ap.ListFeatureClasses()

ap.SpatialReference(102382)
spatial_reference = ap.SpatialReference(102382)

ap.env.workspace = local_geodatabase

In [3]:
# creating necesarry files in local geodatabase
if ap.Exists(local_geodatabase) == False:
    ap.CreateFileGDB_management(os.getcwd(), 'working_files.gdb')
    
for each_network in network_shapefiles:
    network_name = ap.Describe(local_shapefiles_directory + '//' + each_network).baseName

    if ap.Exists('%s_to_line' % network_name) == False:       
        ap.FeatureToLine_management(local_shapefiles_directory + '//%s.shp' % network_name,
                                    '%s_to_line' % network_name)
        
    if ap.Exists('%s_dissolved_to_line' % network_name) == False:       
        ap.Dissolve_management(local_shapefiles_directory + '//%s.shp' % network_name, 
                                   'in_memory\\%s_dissolved'% network_name,
                                   '',
                                   '',
                                   'MULTI_PART',
                                   'DISSOLVE_LINES')
                
        ap.FeatureToLine_management('in_memory\\%s_dissolved' % network_name,
                                            '%s_dissolved_to_line' % network_name)
        
        ap.Delete_management('in_memory\\%s_dissolved' % network_name)

In [4]:
for each_network in network_shapefiles:
    network_name = ap.Describe(local_shapefiles_directory + '\\' + each_network).baseName
    
  
    if ap.Exists('%s_crossings' % network_name) == False:
        ap.CreateFeatureclass_management(local_geodatabase, '%s_crossings' % network_name,
                                         'POINT',
                                         '',
                                         '',
                                         '',
                                         spatial_reference)
    else:
        ap.DeleteRows_management('%s_crossings' % network_name)
                     
        
    for field in ['intersections', 'overpasses', 'dissolved_angles', 'dissolved_ids', 'notes']:

        try:
            ap.AddField_management('%s/%s_crossings' % (local_geodatabase, network_name),
                                       '%s_%s' % (network_name[:3], field),
                                       'STRING')
        except: 
            pass

In [7]:
overpass_dictionary = {}

for each_network in network_shapefiles:
    network_name = ap.Describe(local_shapefiles_directory + '\\' + each_network).baseName
    
    with ap.da.SearchCursor('%s_to_line' % network_name,
                        'FID_%s' % network_name,
                        sql_clause = (None,'ORDER BY FID_%s' % network_name))as overpass_cursor:
        
        overpass_dictionary[network_name] = []
        last_feature_id = None
    
        for each_feature in overpass_cursor:
            feature_id = each_feature[0]
        
            if feature_id == last_feature_id:
                overpass_dictionary[network_name].append(last_feature_id)
                
            else:
                pass
            
            last_feature_id = feature_id
    try:
        ap.MakeFeatureLayer_management('%s_to_line' % network_name,
                                   '%s_to_line_lyr' % network_name)
    except:
        pass

In [8]:
for each_network in network_shapefiles:
    point_dictionary = defaultdict(list)
    network_name = ap.Describe(local_shapefiles_directory + '\\' + each_network).baseName

    with ap.da.SearchCursor('%s_dissolved_to_line' % network_name,
                        ['SHAPE@', 'OID@'],
                        '',
                        spatial_reference
                        ) as angle_cursor:
        
        for each_line in angle_cursor:
            the_lines = of_the_line = each_line[0]
            line_id = each_line[1]
            
            point_dictionary[coordinates_of(the_lines.firstPoint)].append((angle_of_first_10m(of_the_line), line_id)) 
            point_dictionary[coordinates_of(the_lines.lastPoint)].append((angle_of_last_10m(of_the_line), line_id))
 

    select_overpasses = '"FID_%s" IN %s' % (network_name, tuple(overpass_dictionary[network_name]))
    
    ap.SelectLayerByAttribute_management('%s_to_line_lyr' % network_name, 
                                         'NEW_SELECTION',
                                         select_overpasses)
    
    with ap.da.SearchCursor('%s_to_line_lyr' % network_name,
                        ['SHAPE@'],
                        '',
                        spatial_reference) as cursor_for_overpasses:
        
        for each_line in cursor_for_overpasses:
            the_lines = of_the_line = each_line[0]
         
            if coordinates_of(the_lines.firstPoint) in point_dictionary.keys():
                point_dictionary[coordinates_of(the_lines.firstPoint)].append(angle_of_first_10m(of_the_line)) 
    
            
            if coordinates_of(the_lines.lastPoint) in point_dictionary.keys():
                point_dictionary[coordinates_of(the_lines.lastPoint)].append(angle_of_last_10m(of_the_line)) 
                
          
    with ap.da.InsertCursor('%s_crossings' % network_name, 
                           ['SHAPE@XY', 
                            '%s_dissolved_angles' % network_name[:3],
                            '%s_dissolved_ids' % network_name[:3],
                            '%s_intersections' % network_name[:3],
                            '%s_overpasses' % network_name[:3]]) as cursor_for_points_with_angles:
        
        for each_point in point_dictionary.items():
            point = ap.CreateObject("Point")
            point.X, point.Y = float(each_point[0][0]), float(each_point[0][1])
            
            crossings = []
            link_ids = []
            overpasses = []
            intersections = []
            
            for each_link in each_point[1]:        
                try:    
                    if len(each_link) == 2:
                        crossings.append(each_link[0])
                        link_ids.append(each_link[1])
                    
                except:
                    overpasses.append(each_link)
            
            if crossings != overpasses:
                intersections = [x for x in crossings if x not in overpasses]
                
            else:
                intersections = []
                
            cursor_for_points_with_angles.insertRow([point] + 
                
                [str(crossings),
                str(link_ids),
                str(sorted(intersections)), 
                str(sorted(overpasses))])
    
    ap.Delete_management( '%s_to_line_lyr' % network_name)
    del point_dictionary
    del overpass_dictionary[network_name]

del overpass_dictionary

In [12]:
for a in network_shapefiles:
    network_name = ap.Describe(local_shapefiles_directory + '\\' + a).baseName
    try:
        ap.MakeFeatureLayer_management('%s_crossings' % network_name,
                                   '%s_crossings_lyr' % network_name)
    
    except:
        pass
    
try:
    ap.GenerateNearTable_analysis("DTD_2010_Links_directions_crossings_lyr", 
                                  "NavStreets_CDOT_TMC14Q1_WM84_crossings_lyr", 
                                  'DTD_2010_Links_directions_crossings_Near_NavStreets_CDOT_TMC14Q1_WM84_crossings', 
                                  "50 Meters", 
                                  "NO_LOCATION", 
                                  "ANGLE", 
                                  "ALL", 
                                  "0", 
                                  "PLANAR")
except:
    pass

for fields in ['matched_Nav_angles', 
               'matched_DTD_angles', 
               'matched_Nav_angle_to_matched_DTD_angle', 
               'matched_Nav_id_to_matched_DTD_id', 
               'matched_notes']:
    try:
        ap.AddField_management('DTD_2010_Links_directions_crossings_Near_NavStreets_CDOT_TMC14Q1_WM84_crossings', 
                          fields, 
                          'TEXT')
    except:
        pass

ap.JoinField_management("DTD_2010_Links_directions_crossings_Near_NavStreets_CDOT_TMC14Q1_WM84_crossings", 
                        "IN_FID", 
                        "DTD_2010_Links_directions_crossings", 
                        "OBJECTID", 
                        '''DTD_crossings;DTD_crossings_link_id;
                        DTD_intersections;
                        DTD_overpasses;
                        DTD_dissolved_angles;
                        DTD_dissolved_ids''')


ap.JoinField_management("DTD_2010_Links_directions_crossings_Near_NavStreets_CDOT_TMC14Q1_WM84_crossings",
                           "OBJECTID", 
                           "NavStreets_CDOT_TMC14Q1_WM84_crossings",
                           "OBJECTID", 
                           '''Nav_crossings;
                           Nav_crossings_link_id;
                           Nav_intersections;
                           Nav_overpasses;
                           Nav_dissolved_angles;
                           Nav_dissolved_ids''')


for a in network_shapefiles:
    network_name = ap.Describe(local_shapefiles_directory + '\\' + a).baseName
    try:
        ap.Delete_management('%s_crossings_lyr' % network_name)
                                   
    except:
        pass


In [ ]:
network_name

In [ ]:
sorted(each_point[1][: each_point[1].index('i')])


In [ ]:
ap.MakeFeatureLayer_management('%s_to_line' % network_name,
                                   '%s_to_line_lyr' % network_name)

In [ ]:
Counter(sorted(each_point[1][: each_point[1].index('i')]))

In [ ]:
del point_dictionary

In [ ]:
import sys

In [ ]:
sys.getsizeof(point_dictionary)

In [ ]:
[x for x in crossings if x not in overpasses]


In [ ]:
crossings

In [ ]:
overpasses

In [ ]:
sorted(overpasses)


In [10]:
for a in network_shapefiles:
    network_name = ap.Describe(local_shapefiles_directory + '\\' + a).baseName
    try:
        ap.MakeFeatureLayer_management('%s_crossings' % network_name,
                                   '%s_crossings_lyr' % network_name)
    
    except:
        pass

ap.GenerateNearTable_analysis("DTD_2010_Links_directions_crossings_lyr", "NavStreets_CDOT_TMC14Q1_WM84_crossings_lyr", "C:/Users/mcintyrei/Documents/ArcGIS/Default.gdb/DTD_2010_Links_directions_near_Nav", "50 Meters", "NO_LOCATION", "ANGLE", "ALL", "0", "PLANAR")

<Result 'C:\\Users\\mcintyrei\\Documents\\ArcGIS\\Default.gdb\\DTD_2010_Links_directions_near_Nav'>